In [1]:
import os

In [2]:
%pwd

'd:\\2025\\end-to-end-machine-learning-project-with-MLfllow\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\2025\\end-to-end-machine-learning-project-with-MLfllow'

In [32]:
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/dhruvj94/end-to-end-machine-learning-project-with-MLfllow.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="dhruvj94"
os.environ["MLFLOW_TRACKING_PASSWORD"]="5eb30e6c65eea9caa3e228545bb28e7be166334c"

In [33]:
import dagshub
dagshub.init(repo_owner='dhruvj94', repo_name='end-to-end-machine-learning-project-with-MLfllow', mlflow=True)



[2025-03-08 14:48:08,538: INFO: _client: - HTTP Request: GET https://dagshub.com/api/v1/repos/dhruvj94/end-to-end-machine-learning-project-with-MLfllow "HTTP/1.1 200 OK"]


Initialized MLflow to track repo "dhruvj94/end-to-end-machine-learning-project-with-MLfllow"

[2025-03-08 14:48:08,562: INFO: helpers: - Initialized MLflow to track repo "dhruvj94/end-to-end-machine-learning-project-with-MLfllow"]


Repository dhruvj94/end-to-end-machine-learning-project-with-MLfllow initialized!

[2025-03-08 14:48:08,572: INFO: helpers: - Repository dhruvj94/end-to-end-machine-learning-project-with-MLfllow initialized!]


In [34]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    metric_file_name: Path
    target_column: str
    mlflow_uri: str

In [35]:
from mlproject.constatnts import *
from mlproject.utils.common import read_yaml, create_directories, save_json

In [44]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.schema = read_yaml(schema_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        schema = self.schema.TARGET_COLUMN
        params = self.params.ElasticNet
        
        create_directories([config.root_dir])

        model_evaluation_config= ModelEvaluationConfig(
            root_dir = config.root_dir,
            test_data_path = config.test_data_path,
            model_path = config.model_path,
            all_params = params,
            metric_file_name = config.metric_file_name,
            target_column = schema.name,
            mlflow_uri = "https://dagshub.com/dhruvj94/end-to-end-machine-learning-project-with-MLfllow.mlflow"
        )
        return model_evaluation_config

In [45]:
import os
import pandas as pd
import numpy as np
import mlflow
import mlflow.sklearn
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from urllib.parse import urlparse
import joblib

In [48]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    
    def eval_metrics(self,actual, pred):
        rmse = np.sqrt(mean_squared_error(actual, pred))
        mae = mean_absolute_error(actual, pred)
        r2 = r2_score(actual, pred)
        return rmse, mae, r2
    


    def log_into_mlflow(self):

        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)

        test_x = test_data.drop([self.config.target_column], axis=1)
        test_y = test_data[[self.config.target_column]]

        mlflow.set_experiment('TNF_EXP')
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme


        with mlflow.start_run():

            predicted_qualities = model.predict(test_x)

            (rmse, mae, r2) = self.eval_metrics(test_y, predicted_qualities)
            
            # Saving metrics as local
            scores = {"rmse": rmse, "mae": mae, "r2": r2}
            save_json(path=Path(self.config.metric_file_name), data=scores)

            mlflow.log_params(self.config.all_params)

            mlflow.log_metric("rmse", rmse)
            mlflow.log_metric("r2", r2)
            mlflow.log_metric("mae", mae)


            # Model registry does not work with file store
            if tracking_url_type_store != "file":

                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.sklearn.log_model(model, "model", registered_model_name="ElasticnetModel2")
            else:
                mlflow.sklearn.log_model(model, "model")

    


In [49]:
try:
    config = ConfigurationManager()
    model_evaluation_config= config.get_model_evaluation_config()
    model_evaluation_config = ModelEvaluation(config=model_evaluation_config)
    model_evaluation_config.log_into_mlflow()
except Exception as e:
    raise e

[2025-03-08 14:56:51,892: INFO: common: - yaml file: config\config.yaml loaded successfully]
[2025-03-08 14:56:51,905: INFO: common: - yaml file: schema.yaml loaded successfully]
[2025-03-08 14:56:51,916: INFO: common: - yaml file: params.yaml loaded successfully]
[2025-03-08 14:56:51,923: INFO: common: - created directory at: artifacts]
[2025-03-08 14:56:51,928: INFO: common: - created directory at: artifacts/model_evaluation]


2025/03/08 14:56:52 INFO mlflow.tracking.fluent: Experiment with name 'TNF_EXP' does not exist. Creating a new experiment.


[2025-03-08 14:56:54,413: INFO: common: - json file saved at: artifacts\model_evaluation\metrics.json]


2025/03/08 14:58:26 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Successfully registered model 'ElasticnetModel2'.
2025/03/08 14:58:32 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: ElasticnetModel2, version 1
Created version '1' of model 'ElasticnetModel2'.


🏃 View run unleashed-colt-790 at: https://dagshub.com/dhruvj94/end-to-end-machine-learning-project-with-MLfllow.mlflow/#/experiments/3/runs/aff46a4db3e84121a852f3de4b5ac851
🧪 View experiment at: https://dagshub.com/dhruvj94/end-to-end-machine-learning-project-with-MLfllow.mlflow/#/experiments/3
